## Import data

In [ ]:
import pandas as pd
import numpy as np

interval = "1h"
klines_df = pd.read_csv(f"klines_data_{interval}.csv")
signals_df = pd.read_csv("tradingview_signals_server.csv")


signals_df["utc_time"] = pd.to_datetime(signals_df["unix_timestamp"], unit="s", utc=True).dt.floor("H")

print(signals_df[["symbol", "utc_time", "signal"]].head())


print('klines_df.shape', klines_df.shape)
klines_df.info()

klines_df.shape (60510, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60510 entries, 0 to 60509
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    60510 non-null  object 
 1   Open    60510 non-null  float64
 2   High    60510 non-null  float64
 3   Low     60510 non-null  float64
 4   Close   60510 non-null  float64
 5   Volume  60510 non-null  float64
 6   Symbol  60510 non-null  object 
dtypes: float64(5), object(2)
memory usage: 3.2+ MB


In [ ]:
print('signals_df.shape', signals_df.shape)
signals_df.info()

signals_df.shape (32267, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32267 entries, 0 to 32266
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   #               32267 non-null  int64  
 1   id              32267 non-null  int64  
 2   symbol          32267 non-null  object 
 3   timeframe       32267 non-null  object 
 4   signal          32267 non-null  object 
 5   entry_price     32267 non-null  float64
 6   unix_timestamp  32267 non-null  int64  
 7   utc_time        32267 non-null  object 
 8   local_time      32267 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 2.2+ MB


In [ ]:
klines_df['Date'] = pd.to_datetime(klines_df['Date'])
signals_df['utc_time'] = pd.to_datetime(signals_df['utc_time'])

signals_df = signals_df[signals_df["timeframe"] == "1h"]

## Backtesting

In [ ]:
!pip install backtesting
!pip install pandas_ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=93f724f3e3db644c630b52e782649a00b7c35b31a88c6266a8b85d08ba8e23f8
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta


In [ ]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting import Backtest, Strategy



trade_log = []

class SignalStrategy(Strategy):
    stop_loss_multiplier = 0.5
    take_profit_multiplier = 1.0
    signals_df = None
    symbol = None

    def init(self):
        """Load trading signals and ATR indicator"""
        self.signals = self.signals_df[self.signals_df["symbol"] == self.symbol]
        self.used_signals = set()

        def atr_indicator(h, l, c):
            """Calculate ATR (Average True Range)"""
            try:
                atr_values = ta.atr(pd.Series(h), pd.Series(l), pd.Series(c), length=14)
                return atr_values.fillna(0).to_numpy() if atr_values is not None else np.zeros(len(h))
            except Exception as e:
                print(f"Error calculating ATR: {e}")
                return np.zeros(len(h))

        self.atr = self.I(atr_indicator, self.data.High, self.data.Low, self.data.Close)

    def next(self):
        """Process signals and open trades accordingly"""
        current_time = self.data.index[-1]
        current_price = self.data.Close[-1]
        current_atr = self.atr[-1]

        if current_atr == 0 or pd.isna(current_atr):
            return

        for _, signal in self.signals.iterrows():
            signal_time = signal['utc_time']

            if signal_time in self.used_signals:
                continue

            if signal_time in self.data.index:
                self.used_signals.add(signal_time)

                if signal['signal'] == "STRONG_BUY" and not self.position:
                    stop_loss = current_price - (self.stop_loss_multiplier * current_atr)
                    take_profit = current_price + (self.take_profit_multiplier * current_atr)

                    if stop_loss < current_price < take_profit:
                        print(f"🚀 LONG {self.symbol} | {current_time} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
                        self.buy(sl=stop_loss, tp=take_profit)

                elif signal['signal'] == "STRONG_SELL" and not self.position:
                    stop_loss = current_price + (self.stop_loss_multiplier * current_atr)
                    take_profit = current_price - (self.take_profit_multiplier * current_atr)

                    if take_profit < current_price < stop_loss:
                        print(f"🚨 SHORT {self.symbol} | {current_time} | Price: {current_price} | SL: {stop_loss:.4f} | TP: {take_profit:.4f}")
                        self.sell(sl=stop_loss, tp=take_profit)

    def notify_trade(self, trade):
        """Log trade results when a trade is closed"""
        if trade.is_closed:
            trade_log.append({
                "Symbol": self.symbol,
                "Entry Time": trade.entry_time,
                "Exit Time": trade.exit_time,
                "Entry Price": trade.entry_price,
                "Exit Price": trade.exit_price,
                "Return [%]": trade.pl_pct * 100,
            })


results_list = []


for symbol in klines_df["Symbol"].unique():
    print(f"🔍 Testing {symbol}...")

    data = klines_df[klines_df["Symbol"] == symbol].copy()
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index("Date", inplace=True)

    if not isinstance(data.index, pd.DatetimeIndex):
        print(f"⚠️ Index for {symbol} is not DatetimeIndex, attempting to convert...")
        data.index = pd.to_datetime(data.index)

    data.drop(columns=["Symbol"], inplace=True)

    symbol_signals = signals_df[(signals_df["symbol"] == symbol) & (signals_df["timeframe"] == f"{interval}")]
    unique_signals_count = symbol_signals["utc_time"].nunique()

    if symbol_signals.empty:
        print(f"⚠️ No signals for {symbol}, skipping...")
        continue

    print(f"📊 Found {len(symbol_signals)} signals ({unique_signals_count} unique) for {symbol}")

    SignalStrategy.signals_df = signals_df
    SignalStrategy.symbol = symbol

    bt = Backtest(data, SignalStrategy, cash=1_000_000, commission=0.0005)
    stats = bt.run()

    total_profit = sum(t["Return [%]"] for t in trade_log if t["Return [%]"] > 0)
    total_loss = sum(t["Return [%]"] for t in trade_log if t["Return [%]"] < 0)

    results = {
        "Symbol": symbol,
        "Total Trades": stats["# Trades"],
        "Win Rate": stats["Win Rate [%]"],
        "Return [%]": stats["Return [%]"],
        "Total Profit [%]": total_profit,
        "Total Loss [%]": total_loss,
        "Final Return [%]": total_profit + total_loss
    }
    results_list.append(results)

    print(f"✅ {symbol} — Return: {results['Return [%]']:.2f}%, Trades: {results['Total Trades']}, Win Rate: {results['Win Rate']:.2f}%, Profit: {results['Total Profit [%]']:.2f}%, Loss: {results['Total Loss [%]']:.2f}%\n")

results_df = pd.DataFrame(results_list)
results_df.to_csv(f"backtest_results_{interval}.csv", index=False)
trade_log_df = pd.DataFrame(trade_log)
trade_log_df.to_csv("trade_log.csv", index=False)

print("📌 Backtest completed! Results saved in backtest_results.csv")
print("📌 Trade log saved in trade_log.csv")

display(results_df)


     symbol                  utc_time      signal
0   BNBUSDT 2025-03-01 16:00:00+00:00  STRONG_BUY
1   BTCUSDT 2025-03-01 16:00:00+00:00     NEUTRAL
2   ETHUSDT 2025-03-01 16:00:00+00:00        SELL
3  CAKEUSDT 2025-03-01 16:00:00+00:00  STRONG_BUY
4   ETHUSDT 2025-03-01 16:00:00+00:00     NEUTRAL
🔍 Тестируем BNBUSDT...
📊 Найдено 34 сигналов (34 уникальных) для BNBUSDT


<ipython-input-18-e3fbddb4a81a>:13: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  signals_df["utc_time"] = pd.to_datetime(signals_df["unix_timestamp"], unit="s", utc=True).dt.floor("H")


Backtest.run:   0%|          | 0/192 [00:00<?, ?it/s]

🚀 LONG BNBUSDT | 2025-03-01 16:00:00+00:00 | Цена: 606.63 | SL: 604.2745 | TP: 611.3410
🚀 LONG BNBUSDT | 2025-03-02 17:00:00+00:00 | Цена: 631.01 | SL: 627.8793 | TP: 637.2714
🚀 LONG BNBUSDT | 2025-03-02 18:00:00+00:00 | Цена: 627.93 | SL: 624.7032 | TP: 634.3836
🚀 LONG BNBUSDT | 2025-03-02 23:00:00+00:00 | Цена: 623.94 | SL: 620.8114 | TP: 630.1971
🚀 LONG BNBUSDT | 2025-03-03 00:00:00+00:00 | Цена: 620.23 | SL: 617.1286 | TP: 626.4328
🚨 SHORT BNBUSDT | 2025-03-03 09:00:00+00:00 | Цена: 600.15 | SL: 603.6671 | TP: 593.1158
🚨 SHORT BNBUSDT | 2025-03-03 15:00:00+00:00 | Цена: 591.83 | SL: 595.5291 | TP: 584.4319
🚨 SHORT BNBUSDT | 2025-03-03 19:00:00+00:00 | Цена: 573.17 | SL: 577.2951 | TP: 564.9197
🚨 SHORT BNBUSDT | 2025-03-03 20:00:00+00:00 | Цена: 574.74 | SL: 578.8866 | TP: 566.4467
🚨 SHORT BNBUSDT | 2025-03-03 22:00:00+00:00 | Цена: 578.41 | SL: 582.7177 | TP: 569.7946
🚨 SHORT BNBUSDT | 2025-03-04 02:00:00+00:00 | Цена: 558.94 | SL: 563.7585 | TP: 549.3030
🚨 SHORT BNBUSDT | 2025-03-

Backtest.run:   0%|          | 0/192 [00:00<?, ?it/s]

...

🚨 SHORT SSVUSDT | 2025-03-01 17:00:00+00:00 | Цена: 10.26 | SL: 10.3345 | TP: 10.1111
🚨 SHORT SSVUSDT | 2025-03-02 01:00:00+00:00 | Цена: 10.36 | SL: 10.4378 | TP: 10.2043
🚨 SHORT SSVUSDT | 2025-03-02 03:00:00+00:00 | Цена: 10.3 | SL: 10.3758 | TP: 10.1484
🚨 SHORT SSVUSDT | 2025-03-02 04:00:00+00:00 | Цена: 10.39 | SL: 10.4657 | TP: 10.2386
🚨 SHORT SSVUSDT | 2025-03-02 07:00:00+00:00 | Цена: 10.23 | SL: 10.3017 | TP: 10.0865
🚨 SHORT SSVUSDT | 2025-03-02 08:00:00+00:00 | Цена: 10.3 | SL: 10.3716 | TP: 10.1568
🚨 SHORT SSVUSDT | 2025-03-02 14:00:00+00:00 | Цена: 10.2 | SL: 10.2693 | TP: 10.0614
🚨 SHORT SSVUSDT | 2025-03-02 15:00:00+00:00 | Цена: 10.37 | SL: 10.4515 | TP: 10.2070
🚀 LONG SSVUSDT | 2025-03-02 17:00:00+00:00 | Цена: 11.64 | SL: 11.5085 | TP: 11.9031
🚀 LONG SSVUSDT | 2025-03-02 18:00:00+00:00 | Цена: 11.85 | SL: 11.7094 | TP: 12.1312
🚀 LONG SSVUSDT | 2025-03-02 19:00:00+00:00 | Цена: 12.11 | SL: 11.9662 | TP: 12.3977
🚨 SHORT SSVUSDT | 2025-03-03 15:00:00+00:00 | Цена: 10.74 | 

Backtest.run:   0%|          | 0/192 [00:00<?, ?it/s]

🚨 SHORT NEIROUSDT | 2025-03-01 17:00:00+00:00 | Цена: 0.00035466 | SL: 0.0004 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-02 15:00:00+00:00 | Цена: 0.00036156 | SL: 0.0004 | TP: 0.0004
🚀 LONG NEIROUSDT | 2025-03-02 23:00:00+00:00 | Цена: 0.00038055 | SL: 0.0004 | TP: 0.0004
🚨 SHORT NEIROUSDT | 2025-03-03 03:00:00+00:00 | Цена: 0.00036142 | SL: 0.0004 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 06:00:00+00:00 | Цена: 0.00035318 | SL: 0.0004 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 08:00:00+00:00 | Цена: 0.00035662 | SL: 0.0004 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 14:00:00+00:00 | Цена: 0.00033766 | SL: 0.0003 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 15:00:00+00:00 | Цена: 0.0003379 | SL: 0.0003 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 16:00:00+00:00 | Цена: 0.00034089 | SL: 0.0003 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 19:00:00+00:00 | Цена: 0.00031804 | SL: 0.0003 | TP: 0.0003
🚨 SHORT NEIROUSDT | 2025-03-03 22:00:00+00:00 | Цена: 0.00032262 | SL: 0.0003 | TP: 0.0003
🚨

Backtest.run:   0%|          | 0/192 [00:00<?, ?it/s]

🚨 SHORT ACEUSDT | 2025-03-01 17:00:00+00:00 | Цена: 0.91 | SL: 0.9159 | TP: 0.8981
🚨 SHORT ACEUSDT | 2025-03-01 19:00:00+00:00 | Цена: 0.914 | SL: 0.9197 | TP: 0.9025
🚨 SHORT ACEUSDT | 2025-03-02 08:00:00+00:00 | Цена: 0.923 | SL: 0.9286 | TP: 0.9118
🚀 LONG ACEUSDT | 2025-03-02 13:00:00+00:00 | Цена: 0.944 | SL: 0.9381 | TP: 0.9559
🚀 LONG ACEUSDT | 2025-03-02 16:00:00+00:00 | Цена: 1.008 | SL: 0.9987 | TP: 1.0265
🚀 LONG ACEUSDT | 2025-03-02 17:00:00+00:00 | Цена: 1.031 | SL: 1.0208 | TP: 1.0513
🚀 LONG ACEUSDT | 2025-03-02 18:00:00+00:00 | Цена: 1.019 | SL: 1.0087 | TP: 1.0397
🚀 LONG ACEUSDT | 2025-03-02 22:00:00+00:00 | Цена: 1.045 | SL: 1.0355 | TP: 1.0639
🚀 LONG ACEUSDT | 2025-03-03 01:00:00+00:00 | Цена: 1.012 | SL: 1.0027 | TP: 1.0306
🚨 SHORT ACEUSDT | 2025-03-03 19:00:00+00:00 | Цена: 0.87 | SL: 0.8817 | TP: 0.8466
🚨 SHORT ACEUSDT | 2025-03-03 21:00:00+00:00 | Цена: 0.839 | SL: 0.8511 | TP: 0.8148
🚨 SHORT ACEUSDT | 2025-03-03 22:00:00+00:00 | Цена: 0.855 | SL: 0.8668 | TP: 0.8313


In [ ]:
import pandas as pd

klines_df = pd.read_csv("klines_data_1h.csv")
signals_df = pd.read_csv("tradingview_signals_server.csv")
interval = '1h'

df = signals_df[(signals_df["timeframe"] == f"{interval}")]
df.sort_values(by=["symbol", 'utc_time'], inplace=True)
df.to_csv(f"signals_{interval}.csv", index=False)
df

<ipython-input-6-64dd86021997>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by=["symbol", 'utc_time'], inplace=True)


,#,id,symbol,timeframe,signal,entry_price,unix_timestamp,utc_time,local_time
234,235,235,1000CATUSDT,1h,STRONG_SELL,0.009621,1740855634,2025-03-01 19:00:34.000000,2025-03-02 02:00:34.000000
296,297,297,1000CATUSDT,1h,STRONG_SELL,0.009602,1740859218,2025-03-01 20:00:18.000000,2025-03-02 03:00:18.000000
1490,1491,1491,1000CATUSDT,1h,STRONG_BUY,0.010320,1740935011,2025-03-02 17:03:31.000000,2025-03-03 00:03:31.000000
1638,1639,1639,1000CATUSDT,1h,STRONG_BUY,0.010530,1740938482,2025-03-02 18:01:22.000000,2025-03-03 01:01:22.000000
1767,1768,1768,1000CATUSDT,1h,STRONG_BUY,0.010600,1740942037,2025-03-02 19:00:37.000000,2025-03-03 02:00:37.000000
...,...,...,...,...,...,...,...,...,...
27415,27416,27416,ZRXUSDT,1h,STRONG_SELL,0.253400,1741536006,2025-03-09 16:00:06.000000,2025-03-09 23:00:06.000000
28300,28301,28301,ZRXUSDT,1h,STRONG_SELL,0.247200,1741539613,2025-03-09 17:00:13.000000,2025-03-10 00:00:13.000000
29211,29212,29212,ZRXUSDT,1h,STRONG_SELL,0.244000,1741543229,2025-03-09 18:00:29.000000,2025-03-10 01:00:29.000000
30151,30152,30152,ZRXUSDT,1h,STRONG_SELL,0.241900,1741557632,2025-03-09 22:00:32.000000,2025-03-10 05:00:32.000000
